In [1]:
import pypyodbc
import pandas
import datetime
from datetime import date

# Define the Components of the Connection String.
server = '192.168.0.21'
port = '4900'
database = 'ICAM'
username = 'admin'
password = 'Clip_SERENA'

# Variables dates
date_today = datetime.datetime.today()
date_limit = date_today + datetime.timedelta(days=21)
formatted_date_limit = date_limit.strftime('%Y%m%d')
print(formatted_date_limit)

# Create a connection object.
connection_object: pypyodbc.Connection = pypyodbc.connect('DRIVER={HFSQL}; \
                        Server Name =' + server + '; \
                        Server Port=' + port + '; \
                        DATABASE=' + database + '; \
                        UID=' + username + '; \
                        PWD=' + password)
                        #Trusted_Connection=yes;

# Create a Cursor Object, using the connection.
cursor_object: pypyodbc.Cursor = connection_object.cursor()

# Define the Select Query.
sql_select = "SELECT GAMME.GACLEUNIK, \
                CHARGE.DAT as DATE_CHARGE, \
                CHARGRES.DAT as DATE_ORDO, \
                CHARGRES.HEUREDEB as HEURE_ORDO, \
                CLIENT.NOM, \
                AFFAIRE.NAF, \
                AFFAIRE.ETATAF, \
                GAMME.RANG, \
                GAMME.EN_PIECE, \
                GAMME.PHASE, \
                AFFAIRE.QTEAF, \
                GAMME.GA_PREP, \
                GAMME.GA_NBH, \
                GAMME.GA_NBHR, \
                AFFAIRE.TYPEAF, \
                GAMME.COFRAIS \
                FROM GAMME \
                LEFT JOIN AFFAIRE ON GAMME.NAF = AFFAIRE.NAF \
                LEFT JOIN ENGAM ON GAMME.ENCLEUNIK = ENGAM.ENCLEUNIK \
                LEFT JOIN CLIENT ON AFFAIRE.COCLI = CLIENT.COCLI \
                LEFT JOIN CHARGRES ON GAMME.GACLEUNIK = CHARGRES.GACLEUNIK \
                LEFT JOIN CHARGE ON GAMME.GACLEUNIK = CHARGE.GACLEUNIK \
                WHERE GAMME.COFRAIS like 'C35-2' \
                AND DATE_CHARGE <= '"+formatted_date_limit+"'"
                # AND EXISTS ( \
                # SELECT CHARGE.DAT \
                # FROM CHARGE \
                # LEFT JOIN GAMME GAMME2 ON CHARGE.GACLEUNIK = GAMME2.GACLEUNIK \
                # WHERE 1=1 \
                # AND GAMME2.GACLEUNIK = GAMME.GACLEUNIK \
                # AND CHARGE.DAT <= '"+formatted_date_limit+"' \
                # )
# AND AFFAIRE.ETATAF like 'F' \
# CHARGE.DAT as DATE_CHARGE,
# LEFT JOIN CHARGE ON GAMME.GACLEUNIK = CHARGE.GACLEUNIK
cursor_object.execute(sql_select)

# Define the column names.
#columns = [column[0] for column in cursor_object.statistics]
columns = [column[0] for column in cursor_object.description]

# Execute the Query.
# records = cursor_object.fetchmany(10)
records = cursor_object.fetchall()

print(records)

# Dump to a Pandas DataFrame.
ordo_df = pandas.DataFrame.from_records(
    data=records,
    columns=columns
)

# ordo_df

# print the head.
# print(ordo_df.head())

20240703
[]


In [2]:
import json

# Supposons que vous ayez un DataFrame pandas appelé df (ici = donnees_affaires)
# Convertir le DataFrame en une chaîne JSON
json_data = ordo_df.to_json(orient='records')

# Charger la chaîne JSON en tant qu'objet JSON
data = json.loads(json_data)

print(json_data)

[]


In [3]:
ordo_df

,gacleunik
